### 1. Import Dependencies

In [2]:
import pandas as pd

### 2. Load Dataset

In [ ]:
df = pd.read_csv('groceries.csv', on_bad_lines='skip') # Read the CSV file, skipping bad lines

In [8]:
print("raw df shape:", df.shape)

raw df shape: (6105, 4)


In [9]:
print("raw df columns preview:", list(df.columns)[:10])
display(df.head(10))

raw df columns preview: ['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups']


,citrus fruit,semi-finished bread,margarine,ready soups
0,tropical fruit,yogurt,coffee,NaN
1,whole milk,NaN,NaN,NaN
2,pip fruit,yogurt,cream cheese,meat spreads
3,other vegetables,whole milk,condensed milk,long life bakery product
4,rolls/buns,NaN,NaN,NaN
5,pot plants,NaN,NaN,NaN
6,whole milk,cereals,NaN,NaN
7,beef,NaN,NaN,NaN
8,frankfurter,rolls/buns,soda,NaN
9,chicken,tropical fruit,NaN,NaN


In [10]:
def normalize_item(s):
    return s.strip().lower()

In [13]:
# CASE A: single-column where each row is comma-separated item list
if df.shape[1] == 1:
    print("Detected single-column (one transaction per row). Parsing comma-separated items.")
    transactions = df[0].dropna().astype(str).apply(
        lambda x: [normalize_item(i) for i in x.split(',') if i.strip() != ""]
    ).tolist()
    # CASE B: long format (one item per row) with transaction/customer id columns
else:
    cols = [str(c).lower() for c in df.columns]
    print("Detected multi-column CSV. Columns (lowercased):", cols)
    # try to find common column names
    tx_cols = [c for c in df.columns if any(k in str(c).lower() for k in ['transaction','trans','invoice','order','receipt','id'])]
    cust_cols = [c for c in df.columns if any(k in str(c).lower() for k in ['member','customer','cust','user'])]
    item_cols = [c for c in df.columns if any(k in str(c).lower() for k in ['item','product','description','itemdescription','productdescription'])]
    print("possible tx cols:", tx_cols, "possible customer cols:", cust_cols, "possible item cols:", item_cols)

    if tx_cols and item_cols:
        tx_col = tx_cols[0]
        item_col = item_cols[0]
        grouped = df[[tx_col, item_col]].dropna()
        transactions = grouped.groupby(tx_col)[item_col].apply(
            lambda x: [normalize_item(str(i)) for i in x]
        ).tolist()
    elif cust_cols and 'date' in cols and item_cols:
        # common Kaggle long-format: Member_number, Date, itemDescription
        tx_key = [cust_cols[0], [c for c in df.columns if 'date' in str(c).lower()][0]]
        item_col = item_cols[0]
        grouped = df.groupby([cust_cols[0], tx_key[1]])[item_col].apply(lambda x: [normalize_item(str(i)) for i in x])
        transactions = grouped.tolist()
    else:
        # fallback: try treat each row as transaction by joining all non-null columns
        print("Could not confidently detect transaction structure — falling back to join non-null columns per row.")
        transactions = df.fillna('').astype(str).apply(lambda r: [normalize_item(x) for x in r.tolist() if x.strip()!=''], axis=1).tolist()

Detected multi-column CSV. Columns (lowercased): ['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups']
possible tx cols: [] possible customer cols: [] possible item cols: []
Could not confidently detect transaction structure — falling back to join non-null columns per row.


In [14]:
# final basic clean: remove empty tx and remove duplicates within transaction (presence-only)
from collections import Counter


transactions = [[itm for itm in pd.unique(tx)] for tx in transactions if len(tx) > 0]

# Summaries
n_transactions = len(transactions)
all_items = [i for tx in transactions for i in tx]
unique_items = set(all_items)
freq = Counter(all_items)

print(f"\nParsed transactions: {n_transactions}")
print(f"Unique items (parsed): {len(unique_items)}")
print("Top 20 items by frequency:")



Parsed transactions: 6105
Unique items (parsed): 165
Top 20 items by frequency:


/var/folders/7c/gq0bcsjx4z5518py08krjc880000gn/T/ipykernel_37705/1909183862.py:5: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  transactions = [[itm for itm in pd.unique(tx)] for tx in transactions if len(tx) > 0]


In [ ]:
# Display top 20 items by frequency
for item, cnt in freq.most_common(20):
    print(f"  {item}: {cnt}")

  whole milk: 859
  soda: 773
  rolls/buns: 770
  other vegetables: 531
  canned beer: 502
  bottled water: 406
  bottled beer: 406
  yogurt: 384
  pastry: 308
  newspapers: 296
  shopping bags: 296
  sausage: 294
  tropical fruit: 243
  root vegetables: 227
  citrus fruit: 207
  coffee: 206
  brown bread: 195
  pip fruit: 189
  fruit/vegetable juice: 186
  frankfurter: 176


In [18]:
# Rare items
rare_items = [item for item, c in freq.items() if c == 1]

# show items that apear once by name
print("\nItems that appear exactly once:")
for item in rare_items:
    print(f"  {item}")


Items that appear exactly once:
  salad dressing
  toilet cleaner
  kitchen utensil
  ready soups


In [19]:
# show 5 sample transactions
print("\nSample parsed transactions (first 5):")
for i, tx in enumerate(transactions[:5], 1):
    print(i, tx)


Sample parsed transactions (first 5):
1 ['tropical fruit', 'yogurt', 'coffee']
2 ['whole milk']
3 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads']
4 ['other vegetables', 'whole milk', 'condensed milk', 'long life bakery product']
5 ['rolls/buns']
